In [ ]:
import pandas as pd

a = pd.read_csv("mydbvscontigs.txt", sep='\t', engine="python",header=None)
a

In [ ]:
a.columns=["Contigs","Ref","Identity","Alignment_lenght","Mismatches","Gap_opens","q.start","q.end","s.start","s.end","evalue","bitscore"]
del a["Mismatches"]
del a["Gap_opens"]
del a["q.start"]
del a["q.end"]
del a["evalue"]
del a["bitscore"]
a

In [ ]:
b=a["Contigs"].str.split("_")

In [ ]:
c=[int(x[3]) for x in b]

In [ ]:
a.insert(4,"contig_length",c)
a.insert(5,"name",[x[1] for x in b])
a

In [ ]:
a["strand"] = ""

for i in range(a["s.start"].count()):
    if a["s.start"][i]<a["s.end"][i]:
        a["strand"][i]= "+"
    else:
        tmp = a["s.start"][i]
        a["s.start"][i] = a["s.end"][i]
        a["s.end"][i] = tmp
        a["strand"][i]= "-"
    
a   

In [ ]:
(a["Alignment_lenght"]/a["contig_length"]>=0.8).value_counts()

In [ ]:
d=a[a["Alignment_lenght"]/a["contig_length"]>=0.8]
d

In [ ]:
e=d["Ref"].value_counts()

In [ ]:
e.index

In [ ]:
d["Ref"].drop_duplicates()

In [ ]:
e=d[d["Ref"]=="MF474175.1"].sort_values(by=['s.start'])
e

In [ ]:
e

In [ ]:
previous = "0"
for i in range(1, e["s.start"].count()):
    first=e.iloc[i-1]
    second=e.iloc[i]
    if first["s.end"]<second["s.start"]:
        # no overlap
        print(first["name"],"->", second["name"],";")
    else:
        print(previous,"->", second["name"])
    previous = second["name"]

In [ ]:
import graphviz

graphviz1=graphviz.Digraph()
graphviz1.attr(rankdir='LR')

seen={}
def edge(x,y):
    if seen.get((x,y), 0) == 0:
        graphviz1.edge(x, y)
        seen[ (x,y) ] = 1

previous = "0"
prev_strand="+"
for i in range(1, e["s.start"].count()):
    first=e.iloc[i-1]
    second=e.iloc[i]
    if(second["strand"]=="+"):
        graphviz1.node(second["name"]+second["s.start"], second["name"], shape='rarrow')
    else:
        graphviz1.node(second["name"]+second["s.start"], second["name"], shape='larrow')
    if first["s.end"]<second["s.start"]:
        # no overlap
        edge(first["name"]+first["strand"], second["name"]+second["s.start"])
    else:
        edge(previous+prev_strand, second["name"]+second["s.start"])
    previous = second["name"]
    prev_strand = second["s.start"]
graphviz1